In [8]:
%load_ext autoreload
%autoreload 2

from glob import glob
from Data_code.utils import returnMask_test
from transformers import AutoTokenizer, AutoModelForTokenClassification, AdamW, get_linear_schedule_with_warmup
import os
import more_itertools as mit
import json

/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading english - 1grams ...


In [13]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=False)
def get_training_data(dict_map):
    final_output = []
    for key in dict_map.keys():
        print(dict_map[key])
        annotation=dict_map[key]['label']
        text=dict_map[key]['text']
        post_id=key
        tokens_all,attention_masks=returnMask_test(dict_map[key], tokenizer)
        final_output.append([post_id, annotation, tokens_all, attention_masks])
    return final_output

In [10]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]
            
# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id, 
              "end_sentence": -1, 
              "end_token": end, 
              "start_sentence": -1, 
              "start_token": start, 
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_path, save_split):  
    final_output = []
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    if save_split:
#         train_fp = open(save_path+'train.jsonl', 'w')
#         val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')
            
    for tcount, eachrow in enumerate(dataset):
        
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]
        
        if majority_label=='normal':
            continue
        
        print(eachrow[2])
        #all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            print(each_explain)
            explanations.append(list(each_explain))
        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]
        
        print(len(final_explanation))
        print(len(eachrow[2]))
        
        assert(len(final_explanation)==len(eachrow[2]))
        print("==========================")
        
        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)
        
        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')
            
            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))
            test_fp.write(json.dumps(temp)+'\n')
            
#             if post_id in id_division['train']:
#                 train_fp.write(json.dumps(temp)+'\n')
            
#             elif post_id in id_division['val']:
#                 val_fp.write(json.dumps(temp)+'\n')
            
#             elif post_id in id_division['test']:
#                 test_fp.write(json.dumps(temp)+'\n')
#             else:
#                 print(post_id)
    
    if save_split:
        test_fp.close()
        
    return final_output

In [14]:
import json
datasets=['Davidson','Waseem']

for dataset in datasets:
    dataset_path='Dataset/Eraser_eval/'
    save_path=dataset_path+dataset+'/'
    method = 'union'
    save_split=True
    dataset_path='Dataset/Test_data/'+dataset+'_test_new.json'
    with open(dataset_path,'r') as infile:
        dict_map=json.load(infile)
    final_output=get_training_data(dict_map)
    #convert_to_eraser_format(final_output, method, save_path,save_split)

{'text': " @user @user maybe I'm over keen with the ridiculous feminazi brigade. The tweets about rape culture were directly out of sarah!", 'annotation': [[[53, 61]], [[53, 69]]], 'label': 'sexism'}
{'text': ' #BlameOneNotAll because Benedict Cumberbatch, David Tennant, Jared Padalecki, and Jensen Ackles must all be rapists too, right tumblr girls?', 'annotation': [[[102, 120], [128, 140]], [[17, 140]]], 'label': 'sexism'}
{'text': ' @user @user Women must be addressed as milady until they turn you down at which point the correct form of address is "feminazi"', 'annotation': [[[119, 127]], [[72, 128]]], 'label': 'sexism'}
{'text': ' @user @user i ship you 2 angry feminazi\'s together ur hate would b so cute together "i hate this thing" "so do i" "omg yas"', 'annotation': [[[32, 43]], [[15, 42]]], 'label': 'sexism'}
{'text': ' "Fukken anita sarkesian doesnt allow responses to her videos!! Feminazi!!"  http://url  *gets blocked by roguestar, AGAIN*', 'annotation': [[[64, 72], [106, 115]

In [2]:
import glob
test_fp = open('../eraserbenchmark/bash_eraser_test_new.sh', 'w')
#
for dataset in ['Founta','Basile','Davidson','Olid','Waseem']:
    eval_files=glob.glob('../Hate_domain_adaptation/explanation_dicts/'+dataset+'*')
    for file in eval_files:
        str1='PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Hate_domain_adaptation/Dataset/Eraser_eval/'+dataset+'/ --results '+file+' --score_file '+ '../Hate_domain_adaptation/eraser_explanation_dicts/'+file.split('/')[3]
        test_fp.write(str1+'\n')
#     eval_files=glob.glob('../Hate_domain_adaptation/explanation_dicts/'+dataset+'_rationale_BERT_toxic_rationale_10_keyword*')
#     for file in eval_files:
#         str1='PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Hate_domain_adaptation/Dataset/Eraser_eval/'+dataset+'/ --results '+file+' --score_file '+ '../Hate_domain_adaptation/eraser_explanation_dicts/'+file.split('/')[3]
#         test_fp.write(str1+'\n')
test_fp.close()        

In [1]:
import glob
test_fp = open('../eraserbenchmark/bash_eraser_test_shap.sh', 'w')
#
for dataset in ['Basile','Davidson','Founta','Waseem','Mlma','Olid']:
    eval_files=glob.glob('../Hate_domain_adaptation/explanation_dicts_shap/'+dataset+'*')
    for file in eval_files:
        str1='PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Hate_domain_adaptation/Dataset/Eraser_eval/'+dataset+'/ --results '+file+' --score_file '+ '../Hate_domain_adaptation/eraser_explanation_dicts_shap/'+file.split('/')[3]
        test_fp.write(str1+'\n')
#     eval_files=glob.glob('../Hate_domain_adaptation/explanation_dicts/'+dataset+'_rationale_BERT_toxic_rationale_10_keyword*')
#     for file in eval_files:
#         str1='PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test --strict --data_dir ../Hate_domain_adaptation/Dataset/Eraser_eval/'+dataset+'/ --results '+file+' --score_file '+ '../Hate_domain_adaptation/eraser_explanation_dicts/'+file.split('/')[3]
#         test_fp.write(str1+'\n')
test_fp.close()        

In [33]:
files_eraser_eval=glob.glob('../Hate_domain_adaptation/eraser_explanation_dicts_shap/*')

In [34]:
dict_files_group={}

for file  in files_eraser_eval:
    
    if('rationale' in file):
        try:
            dict_files_group[file[:-23]+'_rationale'].append(file)
        except:
            dict_files_group[file[:-23]+'_rationale']=[file]
    if('shap' in file):
        try:
            dict_files_group[file[:-18]].append(file)
        except:
            dict_files_group[file[:-18]]=[file]
    else:
        try:
            dict_files_group[file[:-13]].append(file)
        except:
            dict_files_group[file[:-13]]=[file]

In [35]:
dict_files_group

{'../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert': ['../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2024_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2023_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2025_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2021_50_shap.json'],
 '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd': ['../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd_2025_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd_2021_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd_2023_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd_2022_50_shap.json',
  '../Hate_domain_adaptation/eraser_explanation_dicts_shap/Davidson_bert_cd_2024_50_shap.json'],
 '../Ha

In [36]:
for key in dict_files_group:
    #print(key,len(dict_files_group[key]))
    if(len(dict_files_group[key])<5):
        for ele in sorted(dict_files_group[key]):
            print(ele)
        print("=========================")

../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2021_50_shap.json
../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2023_50_shap.json
../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2024_50_shap.json
../Hate_domain_adaptation/eraser_explanation_dicts_shap/Waseem_bert_2025_50_shap.json


In [38]:
import json


def get_metric_dict(name_of_metric):
    result={}
    for key in dict_files_group:
        
#         if('bert_cd' in key):
#             continue
#         if('rationale' in key):
#             pass
#         else:
#             continue
        
        value=0
        for file in dict_files_group[key]:
            
            
            
            with open(file) as fp:
                output_data = json.load(fp)
            if(name_of_metric=='auprc'):
                value+=output_data['token_soft_metrics']['auprc']
            elif(name_of_metric=='token_f1'):
                value+=output_data['token_prf']['instance_macro']['f1']
            elif(name_of_metric=='iou_f1'):
                value+=output_data['iou_scores'][0]['macro']['f1']
            elif(name_of_metric=='sufficiency'):
                value+=output_data['classification_scores']['sufficiency']
            elif(name_of_metric=='comprehensiveness'):
                value+=output_data['classification_scores']['comprehensiveness']
        result[key[56:]]=value/len(dict_files_group[key])
    result={k: v for k, v in sorted(result.items(), key=lambda item: item[0])}
    return result

In [39]:
name_of_metric='auprc'
results=get_metric_dict(name_of_metric)

for dataset in ['Davidson','Olid','Basile','Founta','Waseem','MLMA']:
    for model in ['bert','bert_cd','Transform_Rationale_CrossAttn_CLS_Drpt_corrected_rationale','Transform_Rationale_SelfAttn_Drpt_corrected_rationale']:
        try:
            print(dataset+'_'+model,results[dataset+'_'+model])
        except KeyError:
            pass

Davidson_bert 0.3718791250609757
Davidson_bert_cd 0.45887803832244944
Olid_bert 0.33859450530026797
Olid_bert_cd 0.37038165787772687
Basile_bert 0.3499836184241008
Basile_bert_cd 0.4646319531471774
Founta_bert 0.37717342043472024
Founta_bert_cd 0.48022494289719486
Waseem_bert 0.4588233935581439
Waseem_bert_cd 0.5060585586909346


In [41]:
name_of_metric='token_f1'
results=get_metric_dict(name_of_metric)

for dataset in ['Davidson','Olid','Basile','Founta','Waseem','MLMA']:
    for model in ['bert','bert_cd','Transform_Rationale_CrossAttn_CLS_Drpt_corrected_rationale','Transform_Rationale_SelfAttn_Drpt_corrected_rationale']:
        try:
            print(dataset+'_'+model,results[dataset+'_'+model])
        except:
            pass


Davidson_bert 0.27879043734043735
Davidson_bert_cd 0.37136456876456875
Olid_bert 0.2815060570173352
Olid_bert_cd 0.3225729859113318
Basile_bert 0.2522912826960009
Basile_bert_cd 0.3482836427137158
Founta_bert 0.254518860608826
Founta_bert_cd 0.3598764077137779
Waseem_bert 0.3175349302937345
Waseem_bert_cd 0.34941564382469953


In [10]:
name_of_metric='iou_f1'
results=get_metric_dict(name_of_metric)

for dataset in ['Davidson','Olid','Basile','Founta','Waseem']:
    for model in ['bert','bert_cd']:
        print(dataset+'_'+model,results[dataset+'_'+model])


Davidson_bert 0.08515643416770688
Davidson_bert_cd 0.14255991607568586
Olid_bert 0.06185808817664673
Olid_bert_cd 0.07676725473897257
Basile_bert 0.10062890108371617
Basile_bert_cd 0.1449496033789166
Founta_bert 0.05719341949586768
Founta_bert_cd 0.06129325358063292
Waseem_bert 0.0078125
Waseem_bert_cd 0.0075


In [11]:
name_of_metric='sufficiency'
results=get_metric_dict(name_of_metric)

for dataset in ['Davidson','Olid','Basile','Founta','Waseem']:
    for model in ['bert','bert_cd']:
        print(dataset+'_'+model,results[dataset+'_'+model])


Davidson_bert 0.028385287392884494
Davidson_bert_cd -0.2956497736182064
Olid_bert -0.17194013442311967
Olid_bert_cd -0.10685329990727561
Basile_bert -0.14387096044421196
Basile_bert_cd -0.3864719812870026
Founta_bert -0.03506338210666881
Founta_bert_cd -0.1401792105506448
Waseem_bert 0.08628971884027124
Waseem_bert_cd -0.010902608055621384


In [12]:
name_of_metric='comprehensiveness'
results=get_metric_dict(name_of_metric)

for dataset in ['Davidson','Olid','Basile','Founta','Waseem']:
    for model in ['bert','bert_cd']:
        print(dataset+'_'+model,results[dataset+'_'+model])


Davidson_bert 0.03512217603623866
Davidson_bert_cd -0.14006396474316718
Olid_bert -0.14291966323341643
Olid_bert_cd -0.09816070760999408
Basile_bert -0.20817292173206808
Basile_bert_cd -0.3802526766061783
Founta_bert 0.008470927222686659
Founta_bert_cd -0.09938222559935908
Waseem_bert -0.014074154570698744
Waseem_bert_cd -0.09447050668299198
